In [46]:
import math as m

def getWDL(score):
    return (score[0] > score[1]) - (score[0] < score[1])

def difference_factor(correct, voorspeld):
    doelsaldo_delta = abs(abs(correct[0]-correct[1]) - abs(voorspeld[0]-voorspeld[1]))
    doelpunten_delta = abs((correct[0]+correct[1]) - (voorspeld[0]+voorspeld[1]))
    return m.pi**(-2*(
        doelsaldo_delta**2/max(abs(voorspeld[0]-voorspeld[1]),2)
        +doelpunten_delta**2/max(abs(voorspeld[0]+voorspeld[1]),2)
      ))


def calc_bonuspoints(correct, voorspeld):
    if (correct[0] - correct[1]) * (voorspeld[0] - voorspeld[1]) < 0:
        return 0
    doelpunten = [2, 1, 3, 7, 10, 17, 27, 44, 71, 115, 186]
    totaal_doelpunten = [0,0,0,1,3,6,10,15,21,28,36,45,55,66,78,91,105,120,136,153,171]
    thuis = doelpunten[correct[0]]
    uit = doelpunten[correct[1]]
    totaal = totaal_doelpunten[correct[0]+correct[1]]
    return m.floor(difference_factor(correct, voorspeld) * (thuis + uit + totaal))

def calc_basepoints(correct, voorspeld):
    if getWDL(correct) != getWDL(voorspeld):
        return 0
    return 7 if getWDL(correct) == 0 else 5

def calc_points(correct, voorspeld):
    return calc_basepoints(correct, voorspeld) + calc_bonuspoints(correct, voorspeld)  


In [47]:
teams = ['ARG', 'AUS', 'BEL', 'BRA', 'CAN', 'CMR', 'CRC', 'CRO', 'DEN', 'ECU', 'ENG', 'ESP', 'FRA', 'GER', 'GHA', 'IRN', 'JPN', 'KOR', 'KSA', 'MAR', 'MEX', 'NED', 'POL', 'POR', 'QAT', 'SEN', 'SRB', 'SUI', 'TUN', 'URU', 'USA', 'WAL']

In [48]:
import json
with open("data_kuleuven.json", "r") as f:
    k = f.read()
data = json.loads(k)
print(data["NED-QAT"])


[[5.523708680762211, 1.7720753024918359, 0.9288295976376811, 0.3245627392299642, 0.08505944359792242, 0.02148491100307942], [4.871115905383919, 9.022767226707465, 2.7470095108221346, 0.9722131553864098, 0.2580205748204244, 0.06615008974422064], [7.018246653625766, 7.551034485718895, 5.993321004855852, 1.455408983955826, 0.3910965856123104, 0.10175806287551252], [6.741204965196053, 7.346053293042494, 4.0006572184697164, 2.098660434959765, 0.3949632076426702, 0.10427852733891402], [4.856324516534342, 5.359116651002198, 2.955130101580551, 1.0856827359900099, 0.4613509172810382, 0.08008816010186649], [4.977750488944864, 5.608749406635435, 3.157601677398483, 1.1842765142272276, 0.3328993740262128, 0.13026673983908918]]


In [52]:
def result_prob(prob, ngoals_home, ngoals_away, nmax=9):
        factor_home = 1
        factor_away = 1
        if ngoals_home >= 5:
            factor_home = [0.5**(i) for i in range(1,nmax-5+2)][ngoals_home-5]
        if ngoals_away >= 5:
            factor_away = [0.5**(i) for i in range(1,nmax-5+2)][ngoals_away-5]
        return prob[min(ngoals_home,5)][min(ngoals_away,5)]*factor_home*factor_away

def getOptimalPronostiek(home: str, away: str):            
    prob = data.get(f"{home}-{away}", None)
    if prob is None:
        return {(k[1],k[0]): v for k,v in getOptimalPronostiek(away, home).items()}
    scores = {}
    best_prono = (0, 0)
    for i in range(10):
        for j in range(10):
            voorspeld = (i, j)
            score = 0
            for k in range(10):
                for l in range(10):
                    correct = (k, l)
                    score += calc_points(correct, voorspeld)*result_prob(prob, k, l)
            scores[voorspeld] = score
    return scores
            
    

In [56]:
sorted(list(getOptimalPronostiek("BEL","CAN").items()), reverse=True, key=lambda n:n[1])


[((4, 1), 424.20021894141513),
 ((5, 1), 411.89928875546894),
 ((4, 2), 401.4601914560084),
 ((3, 2), 398.7979713743463),
 ((5, 2), 392.5009767890406),
 ((6, 1), 392.49991085735155),
 ((3, 1), 386.42773475098704),
 ((7, 1), 381.9560761656041),
 ((4, 0), 381.6225349807657),
 ((6, 2), 376.54603000852626),
 ((4, 3), 375.3502457933019),
 ((5, 0), 369.3995592647309),
 ((7, 2), 367.7689501437416),
 ((8, 1), 365.90806972269206),
 ((3, 0), 361.35294899682066),
 ((6, 0), 359.4904914967272),
 ((8, 2), 354.5528825008594),
 ((7, 0), 352.735713852991),
 ((5, 3), 346.5056437897473),
 ((8, 0), 342.3011452821299),
 ((6, 3), 339.8024711448816),
 ((9, 1), 338.2858074143578),
 ((2, 1), 335.99415725708485),
 ((7, 3), 332.59809706224473),
 ((9, 2), 332.4821847204152),
 ((8, 3), 325.33592929504704),
 ((9, 0), 323.3961606716657),
 ((2, 0), 317.8979662239403),
 ((5, 4), 316.768093725874),
 ((9, 3), 313.9849953229597),
 ((6, 4), 305.6536222464352),
 ((7, 4), 304.2617597489701),
 ((8, 4), 301.4142940418696),
 (

In [31]:
print([0.5**(i) for i in range(1,5+1)])
sum([0.5**(i) for i in range(1,5+1)])

[0.5, 0.25, 0.125, 0.0625, 0.03125]


0.96875